In [1]:
import nimble as nm
#nm.print_all_ipython_imports()
from glue import qglue
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
pd.options.display.mpl_style = 'default'
%matplotlib inline
from pandas.tools.plotting import scatter_matrix, scatter_plot
import seaborn as sb
import datetime as dt
from IPython.display import display
import os
from __future__ import print_function
import math

In [2]:
for root,dir,files in os.walk('..\competition_data'):
    print(root)
    print(files)
    
all_table_dfs = {}
for rdb_table in files:
    #print(rdb_table)
    all_table_dfs[rdb_table[:-4]] = pd.read_csv(os.path.join(root,rdb_table))
    all_table_dfs[rdb_table[:-4]].tname = rdb_table[:-4]

print([df.tname for df in all_table_dfs.values()])
    
for df in all_table_dfs.values():
    if 'tube_assembly_id' in df.columns:
        index_col = 'tube_assembly_id'
    if 'end_form_id' in df.columns:
        index_col = 'end_form_id'
    if 'component_id' in df.columns:
        index_col = 'component_id'
    if df.tname == 'type_component':
        index_col = 'component_type_id'
    if df.tname == 'type_connection':
        index_col = 'connection_type_id'
    df.index = df[index_col]
    df.drop(index_col,1,inplace=True)
    #print(df.info())

..\competition_data
['bill_of_materials.csv', 'components.csv', 'comp_adaptor.csv', 'comp_boss.csv', 'comp_elbow.csv', 'comp_float.csv', 'comp_hfl.csv', 'comp_nut.csv', 'comp_other.csv', 'comp_sleeve.csv', 'comp_straight.csv', 'comp_tee.csv', 'comp_threaded.csv', 'specs.csv', 'test_set.csv', 'train_set.csv', 'tube.csv', 'tube_end_form.csv', 'type_component.csv', 'type_connection.csv', 'type_end_form.csv']
['type_end_form', 'bill_of_materials', 'type_connection', 'comp_hfl', 'train_set', 'comp_nut', 'tube', 'comp_adaptor', 'tube_end_form', 'type_component', 'comp_elbow', 'comp_sleeve', 'comp_other', 'comp_threaded', 'components', 'comp_straight', 'comp_boss', 'comp_float', 'specs', 'comp_tee', 'test_set']


In [3]:
df_list = []
for i in range(1,9):
    df_list.append(all_table_dfs['bill_of_materials'][['component_id_'+str(i),'quantity_'+str(i)]])
    df_list[-1].columns = ['component_id','quantity']
tube_comps = pd.concat(df_list,axis=0)

In [4]:
tube_comps.fillna(value=0,inplace=True)
tube_comps.describe()

,quantity
count,169584.000000
mean,0.342544
std,0.668819
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,6.000000


In [5]:
comp_qtys = tube_comps.groupby('component_id').sum()

In [6]:
comp_qtys.join(all_table_dfs['components'])

,quantity,name,component_type_id
component_id,,,
0,8,NaN,NaN
9999,2,OTHER,OTHER
C-0001,269,SLEEVE,CP-024
C-0002,52,SLEEVE,CP-024
C-0003,48,SLEEVE-FLARED,CP-024
C-0004,62,NUT,CP-026
C-0005,1,ADAPTER-STR,CP-028
C-0006,1,ADAPTER-STR,CP-028
C-0007,23,CONNECTOR-FLARE,CP-014


In [3]:
for df in all_table_dfs.values():
    print(df.tname+':  '+ df.index.name)
    
display(all_table_dfs['train_set'].head())
masterDf = pd.concat([all_table_dfs['train_set'],all_table_dfs['test_set']])
masterDf = masterDf.join(all_table_dfs['tube'])
display(masterDf.head().T)
print('masterDf.columns'), print(masterDf.columns)

type_end_form:  end_form_id
bill_of_materials:  tube_assembly_id
type_connection:  connection_type_id
comp_hfl:  component_id
train_set:  tube_assembly_id
comp_nut:  component_id
tube:  tube_assembly_id
comp_adaptor:  component_id
tube_end_form:  end_form_id
type_component:  component_type_id
comp_elbow:  component_id
comp_sleeve:  component_id
comp_other:  component_id
comp_threaded:  component_id
components:  component_id
comp_straight:  component_id
comp_boss:  component_id
comp_float:  component_id
specs:  tube_assembly_id
comp_tee:  component_id
test_set:  tube_assembly_id


,supplier,quote_date,annual_usage,min_order_quantity,bracket_pricing,quantity,cost
tube_assembly_id,,,,,,,
TA-00002,S-0066,2013-07-07,0,0,Yes,1,21.905933
TA-00002,S-0066,2013-07-07,0,0,Yes,2,12.341214
TA-00002,S-0066,2013-07-07,0,0,Yes,5,6.601826
TA-00002,S-0066,2013-07-07,0,0,Yes,10,4.687770
TA-00002,S-0066,2013-07-07,0,0,Yes,25,3.541561


tube_assembly_id,TA-00001,TA-00001,TA-00001,TA-00001,TA-00001
annual_usage,0,0,0,0,0
bracket_pricing,Yes,Yes,Yes,Yes,Yes
cost,NaN,NaN,NaN,NaN,NaN
id,1,2,3,4,5
min_order_quantity,0,0,0,0,0
quantity,1,2,5,10,25
quote_date,2013-06-23,2013-06-23,2013-06-23,2013-06-23,2013-06-23
supplier,S-0066,S-0066,S-0066,S-0066,S-0066
material_id,SP-0035,SP-0035,SP-0035,SP-0035,SP-0035
diameter,12.7,12.7,12.7,12.7,12.7


masterDf.columns
Index([u'annual_usage', u'bracket_pricing', u'cost', u'id',
       u'min_order_quantity', u'quantity', u'quote_date', u'supplier',
       u'material_id', u'diameter', u'wall', u'length', u'num_bends',
       u'bend_radius', u'end_a_1x', u'end_a_2x', u'end_x_1x', u'end_x_2x',
       u'end_a', u'end_x', u'num_boss', u'num_bracket', u'other'],
      dtype='object')


(None, None)

## FEATURE ENGINEERING STARTS...

In [7]:
all_table_dfs['specs']['num-fea_spec_count'] = all_table_dfs['specs'].count(axis=1)
masterDf = masterDf.join(all_table_dfs['specs'])
masterDf.head().T

tube_assembly_id,TA-00001,TA-00001,TA-00001,TA-00001,TA-00001
annual_usage,0,0,0,0,0
bracket_pricing,Yes,Yes,Yes,Yes,Yes
cost,NaN,NaN,NaN,NaN,NaN
id,1,2,3,4,5
min_order_quantity,0,0,0,0,0
quantity,1,2,5,10,25
quote_date,2013-06-23,2013-06-23,2013-06-23,2013-06-23,2013-06-23
supplier,S-0066,S-0066,S-0066,S-0066,S-0066
material_id,SP-0035,SP-0035,SP-0035,SP-0035,SP-0035
diameter,12.7,12.7,12.7,12.7,12.7


In [8]:
masterDf['dfea-end_a_forming'] = masterDf.join(all_table_dfs['tube_end_form'],on='end_a')['forming']
masterDf['dfea-end_x_forming'] = masterDf.join(all_table_dfs['tube_end_form'],on='end_x')['forming']
masterDf = masterDf.join(all_table_dfs['bill_of_materials'])

comps_df = pd.concat([all_table_dfs[key] for key in all_table_dfs if key.startswith('comp_')])

qty_cols = [col for col in masterDf.columns if col.startswith('quantity_')]

for qty_col in qty_cols:
    masterDf[qty_col].fillna(value=0,inplace=True)

masterDf['num_comps'] = masterDf[qty_cols].sum(axis=1)

comp_numeric_cols = ['weight','thickness','overall_length','bolt_pattern_wide','bolt_pattern_long']

for colname in comp_numeric_cols:
    for i in range(1,9):
        masterDf[colname+'_'+str(i)] = masterDf.join(comps_df,on='component_id_'+str(i),rsuffix='_comp')[colname]
        masterDf[colname+'_'+str(i)].fillna(value=0,inplace=True)
    masterDf['sum_comp_'+colname] = sum([masterDf[colname+'_'+str(i)]*masterDf['quantity_'+str(i)] for i in range(1,9)])
    masterDf['mean_comp_'+colname] = (masterDf['sum_comp_'+colname] / masterDf['num_comps']).fillna(0)
    
comp_binary_cols = ['groove','orientation','unique_feature']
for bincol in comp_binary_cols:
    colname = 'n_'+bincol 
    comps_df[colname] = 0
    comps_df.loc[comps_df[bincol] == 'Yes',colname] = 1
    for i in range(1,9):
        masterDf[colname+'_'+str(i)] = masterDf.join(comps_df,on='component_id_'+str(i),rsuffix='_comp')[colname]
        masterDf[colname+'_'+str(i)].fillna(value=0,inplace=True)
    masterDf['comps_with_'+bincol] = sum([masterDf[colname+'_'+str(i)]*masterDf['quantity_'+str(i)] for i in range(1,9)])
    

In [6]:
#masterDf.join(all_table_dfs['specs'])

display(masterDf.columns)

masterDf.nfea-num_comps.head()
masterDf['dfea-'+'bracket_pricing'] = pd.get_dummies(masterDf.bracket_pricing)['Yes']

Index([u'annual_usage', u'bend_radius', u'bracket_pricing', u'cost',
       u'diameter', u'end_a', u'end_a_1x', u'end_a_2x', u'end_x', u'end_x_1x', 
       ...
       u'comps_with_orientation', u'n_unique_feature_1', u'n_unique_feature_2',
       u'n_unique_feature_3', u'n_unique_feature_4', u'n_unique_feature_5',
       u'n_unique_feature_6', u'n_unique_feature_7', u'n_unique_feature_8',
       u'comps_with_unique_feature'],
      dtype='object', length=130)

In [7]:
masterDf.quote_date = pd.to_datetime(masterDf.quote_date)
masterDf['dfea-quote_month'] = masterDf.quote_date.apply(lambda x: x.month)
#masterDf['quote_year'] = masterDf.quote_date.apply(lambda x: x.year)
print(np.sort(masterDf.quote_month.unique()))

[ 1  2  3  4  5  6  7  8  9 10 11 12]


### Create Categorical Variables here...

In [8]:
def add_dummies(df,colname):
    col_dummies = pd.get_dummies(df[colname])
    df[['dfea-'+colname+'_'+ col for col in col_dummies.columns]] = col_dummies

In [9]:
comp_id_cols = ['component_id_'+str(i) for i in range(1,9)]
for comp_id_col in comp_id_cols:
    freq_IDs = ','.join(masterDf[comp_id_col].value_counts()[:30].index)
    print(freq_IDs)
    masterDf[comp_id_col] = masterDf[comp_id_col].apply(lambda x: x if str(x) in freq_IDs else np.nan)
    print(masterDf[comp_id_col],masterDf[comp_id_col].count())
    #masterDf.loc[masterDf[comp_id_col]  freq_IDs,comp_id_col] = np.nan
    add_dummies(masterDf,comp_id_col)
masterDf.columns.tolist()

C-1621,C-1312,C-1622,C-1624,C-1623,C-1625,C-1620,C-1243,C-1845,C-0448,C-0823,C-1229,C-1420,C-0444,C-1640,C-1313,C-1642,C-1638,C-1848,C-1435,C-1641,C-1639,C-1637,C-1631,C-1649,C-1647,C-1629,C-1547,C-0002,C-1645
tube_assembly_id
TA-00001    C-1622
TA-00001    C-1622
TA-00001    C-1622
TA-00001    C-1622
TA-00001    C-1622
TA-00001    C-1622
TA-00001    C-1622
TA-00001    C-1622
TA-00002    C-1312
TA-00002    C-1312
TA-00002    C-1312
TA-00002    C-1312
TA-00002    C-1312
TA-00002    C-1312
TA-00002    C-1312
TA-00002    C-1312
TA-00003    C-1312
TA-00003    C-1312
TA-00003    C-1312
TA-00003    C-1312
TA-00003    C-1312
TA-00003    C-1312
TA-00003    C-1312
TA-00003    C-1312
TA-00004    C-1312
TA-00004    C-1312
TA-00004    C-1312
TA-00004    C-1312
TA-00004    C-1312
TA-00004    C-1312
             ...  
TA-21182    C-1845
TA-21182    C-1845
TA-21182    C-1845
TA-21183    C-1243
TA-21183    C-1243
TA-21183    C-1243
TA-21183    C-1243
TA-21183    C-1243
TA-21184       NaN
TA-21185     

In [11]:
masterDf[['month_'+str(i) for i in range(1,13)]] = pd.get_dummies(masterDf.quote_month)
masterDf['dfea-end_a_forming'] = pd.get_dummies(masterDf.end_a_forming)['Yes']
masterDf['dfea-end_x_forming'] = pd.get_dummies(masterDf.end_x_forming)['Yes']


mat_dummies = pd.get_dummies(masterDf.material_id)
masterDf[mat_dummies.columns] = mat_dummies

end_col_list = ['end_a_1x', 'end_a_2x', 'end_x_1x', 'end_x_2x']
for col in end_col_list:
    masterDf['dfea-'+col] = pd.get_dummies(masterDf[col])['Y']
masterDf['nfea-numof_small_ends'] =  masterDf[['dfea-'+col for col in end_col_list]].sum(axis = 1)
masterDf.columns

Index([u'annual_usage', u'bend_radius', u'bracket_pricing', u'cost',
       u'diameter', u'end_a', u'end_a_1x', u'end_a_2x', u'end_x', u'end_x_1x', 
       ...
       u'SP-0034', u'SP-0035', u'SP-0036', u'SP-0037', u'SP-0038', u'SP-0039',
       u'SP-0041', u'SP-0046', u'SP-0048', u'num_small_ends'],
      dtype='object', length=333)

In [12]:
from sklearn.cross_validation import train_test_split

masterDf['target'] = masterDf.cost.apply(lambda x: math.log(x+1))
display(masterDf.columns)

Index([u'annual_usage', u'bend_radius', u'bracket_pricing', u'cost',
       u'diameter', u'end_a', u'end_a_1x', u'end_a_2x', u'end_x', u'end_x_1x', 
       ...
       u'SP-0035', u'SP-0036', u'SP-0037', u'SP-0038', u'SP-0039', u'SP-0041',
       u'SP-0046', u'SP-0048', u'num_small_ends', u'target'],
      dtype='object', length=334)

In [13]:
train_vals = masterDf.loc[masterDf.id.isnull(),:]
rel_cols = ['min_order_quantity','quantity','num_comps','diameter','wall','length','num_bends','bend_radius',
            #'end_a_1x', 'end_a_2x', 'end_x_1x', 'end_x_2x', 
            'end_a_forming', 'end_x_forming','bracket_pricing',
           'min_order_quantity','other','num_small_ends', 'spec_count','num_bracket','other','num_boss']
rel_cols.extend([mon_col for mon_col in masterDf.columns if mon_col.startswith('month_')])
rel_cols.extend(['sum_comp_'+colname for colname in comp_numeric_cols])
rel_cols.extend(['mean_comp_'+colname for colname in comp_numeric_cols])
rel_cols.extend(['comps_with_'+bincol for bincol in comp_binary_cols])
rel_cols.extend(mat_dummies.columns)
rel_cols.extend(qty_cols)
rel_cols.extend([col for col in masterDf.columns if col.startswith('dummy_')])

#==========================================================================================
X = train_vals[rel_cols].values
Y = train_vals.target.values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.20)#, random_state=42)

In [19]:
from sklearn.ensemble import ExtraTreesRegressor
et_reg = ExtraTreesRegressor(n_estimators=200,verbose=1,bootstrap=True)

et_reg.fit(X_train,Y_train)
print([fea+': '+str(imp) for fea,imp in zip(rel_cols,et_reg.feature_importances_)])
pd.DataFrame({'Column_Name':rel_cols, 'importance':et_reg.feature_importances_}).to_clipboard()
pY = et_reg.predict(X_test)
rmsle = math.sqrt(np.mean((Y_test - pY)**2))
print('RMSLE with 20% TRAIN DATA: ', rmsle)

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.6s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:   50.9s
[Parallel(n_jobs=1)]: Done 200 jobs       | elapsed:  3.5min
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:  3.5min finished
[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 200 jobs       | elapsed:    0.7s


['min_order_quantity: 0.0170954506553', 'quantity: 0.392629355563', 'num_comps: 0.00645822017854', 'diameter: 0.0828691293891', 'wall: 0.0190834003923', 'length: 0.0244156293515', 'num_bends: 0.0130157909036', 'bend_radius: 0.0162110144107', 'end_a_forming: 0.0147596651507', 'end_x_forming: 0.0111173576499', 'bracket_pricing: 0.0327677968231', 'min_order_quantity: 0.0165167471161', 'other: 0.000858759369076', 'num_small_ends: 0.013247432317', 'spec_count: 0.0209138705705', 'num_bracket: 0.000907333322596', 'other: 0.000877928208486', 'num_boss: 0.00436990835286', 'month_1: 0.0069225492682', 'month_2: 0.00512430987852', 'month_3: 0.00394886139738', 'month_4: 0.00644529411897', 'month_5: 0.00484437266525', 'month_6: 0.00491533038953', 'month_7: 0.00446074019903', 'month_8: 0.00736126615303', 'month_9: 0.00446587227268', 'month_10: 0.00568064510285', 'month_11: 0.00308160889064', 'month_12: 0.00254292696424', 'sum_comp_weight: 0.0207236907895', 'sum_comp_thickness: 0.0141527600532', 'sum_

[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.7s finished


In [15]:
#  ExtraTrees Final Submission
test_vals = masterDf.loc[masterDf.id.notnull(),:]
kaggle_X_test = test_vals[rel_cols].values

et_reg.fit(X,Y)
kaggle_Y_test = et_reg.predict(kaggle_X_test)
result_df = test_vals[['id','cost']]
result_df['cost'] = np.exp(kaggle_Y_test) - 1
result_df.info()
result_df.loc[:,'id'] = result_df.loc[:,'id'].astype(int)
result_df.to_csv('cat_submit_ExtraTreesRegressor.csv', index=False)

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    1.1s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:   58.7s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  2.0min finished
[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  50 jobs       | elapsed:    0.7s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.5s finished
C:\Python27\lib\site-packages\IPython\kernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Python27\lib\site-packages\pandas\core\indexing.py:490: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas

<class 'pandas.core.frame.DataFrame'>
Index: 30235 entries, TA-00001 to TA-21199
Data columns (total 2 columns):
id      30235 non-null float64
cost    30235 non-null float64
dtypes: float64(2)
memory usage: 590.5+ KB


In [18]:
from sklearn.ensemble import GradientBoostingRegressor
gb_reg = GradientBoostingRegressor(n_estimators=500,verbose=1, max_depth=10)

gb_reg.fit(X_train,Y_train)
print([fea+': '+str(imp) for fea,imp in zip(rel_cols,gb_reg.feature_importances_)],sep='\n')
pY = gb_reg.predict(X_test)
rmsle = math.sqrt(np.mean((Y_test - pY)**2))
print('RMSLE with 20% TRAIN DATA: ', rmsle)

      Iter       Train Loss   Remaining Time 
         1           0.5689           26.48m
         2           0.4789           28.10m
         3           0.4060           26.68m
         4           0.3456           25.88m
         5           0.2960           25.54m
         6           0.2552           25.18m
         7           0.2218           24.90m
         8           0.1943           24.66m
         9           0.1715           24.56m
        10           0.1527           24.32m
        20           0.0688           23.80m
        30           0.0472           22.91m
        40           0.0383           21.86m
        50           0.0330           20.67m
        60           0.0302           19.31m
        70           0.0281           18.04m
        80           0.0261           17.15m
        90           0.0241           16.42m
       100           0.0226           15.60m
       200           0.0115           10.72m
       300           0.0063            7.21m
       40

In [20]:
test_vals = masterDf.loc[masterDf.id.notnull(),:]
kaggle_X_test = test_vals[rel_cols].values

gb_reg.fit(X,Y)
kaggle_Y_test = gb_reg.predict(kaggle_X_test)
result_df = test_vals[['id','cost']]
result_df['cost'] = np.exp(kaggle_Y_test) - 1
result_df.info()
result_df.loc[:,'id'] = result_df.loc[:,'id'].astype(int)
result_df.to_csv('cat_submit_GB_Regressor.csv', index=False)

      Iter       Train Loss   Remaining Time 
         1           0.5685           30.61m
         2           0.4794           32.57m
         3           0.4063           33.01m
         4           0.3468           32.02m
         5           0.2980           32.02m
         6           0.2578           32.44m
         7           0.2249           32.83m
         8           0.1972           33.12m
         9           0.1746           32.94m
        10           0.1556           33.03m
        20           0.0708           32.70m
        30           0.0499           30.65m
        40           0.0410           29.01m
        50           0.0362           26.55m
        60           0.0330           24.08m
        70           0.0307           21.79m
        80           0.0292           20.22m
        90           0.0275           18.94m
       100           0.0259           17.66m
       200           0.0146           10.95m
       300           0.0076            6.97m
       40

C:\Python27\lib\site-packages\IPython\kernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
